# 自然言語処理
RNN(Recurrent Neural Network)を使用して、足し算の計算を学習させる。

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense, Activation, RepeatVector
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

Using TensorFlow backend.


In [2]:
np.random.seed(0)

In [3]:
def n(digits=3):
    number = ''
    for i in range(np.random.randint(1, digits + 1)):
        number += np.random.choice(list('0123456789'))
    return int(number)


def padding(chars, maxlen):
    return chars + ' ' * (maxlen - len(chars))

In [4]:
'''
データの生成
'''
N = 20000
N_train = int(N * 0.9)
N_validation = N - N_train

digits = 3  # 最大の桁数
input_digits = digits * 2 + 1  # 例： 123+456
output_digits = digits + 1  # 500+500 = 1000 以上で４桁になる

added = set()
questions = []
answers = []

while len(questions) < N:
    a, b = n(), n()  # 適当な数を２つ生成

    pair = tuple(sorted((a, b)))
    if pair in added:
        continue

    question = '{}+{}'.format(a, b)
    question = padding(question, input_digits)
    answer = str(a + b)
    answer = padding(answer, output_digits)

    added.add(pair)
    questions.append(question)
    answers.append(answer)

chars = '0123456789+ '
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

X = np.zeros((len(questions), input_digits, len(chars)), dtype=np.integer)
Y = np.zeros((len(questions), digits + 1, len(chars)), dtype=np.integer)

for i in range(N):
    for t, char in enumerate(questions[i]):
        X[i, t, char_indices[char]] = 1
    for t, char in enumerate(answers[i]):
        Y[i, t, char_indices[char]] = 1

X_train, X_validation, Y_train, Y_validation = \
    train_test_split(X, Y, train_size=N_train)

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [5]:
'''
モデル設定
'''
n_in = len(chars)
n_hidden = 128
n_out = len(chars)

model = Sequential()

# Encoder
model.add(LSTM(n_hidden, input_shape=(input_digits, n_in)))

# Decoder
model.add(RepeatVector(output_digits))
model.add(LSTM(n_hidden, return_sequences=True))

model.add(TimeDistributed(Dense(n_out)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999),
              metrics=['accuracy'])

## 学習
"a+b"という文字列を入力データ、"t"("t"は、a+bの値を文字で表現したもの)を教師データとして与えて学習させる。２０エポックごとに、学習済モデルを使用して予測された結果を表示している。足し算の計算方法に関する知識はもちろん、"a"や"b"が数字であることや、"+"や"="の記号の意味すら教えていないにも関わらず、学習が進むにつれて、入力データと教師データの組の集合から、足し算の結果を予測できていることが分かる。

In [6]:
'''
モデル学習
'''
epochs = 200
batch_size = 200
sampling = 20

for epoch in range(epochs // sampling):
    model.fit(X_train, Y_train, batch_size=batch_size, epochs=sampling,
              validation_data=(X_validation, Y_validation))

    print("\nEpoch: {0}/{1}".format((epoch+1) * sampling, epochs))
    # 検証データからランダムに問題を選んで答え合わせ
    for i in range(10):
        index = np.random.randint(0, N_validation)
        question = X_validation[np.array([index])]
        answer = Y_validation[np.array([index])]
        prediction = model.predict_classes(question, verbose=0)

        question = question.argmax(axis=-1)
        answer = answer.argmax(axis=-1)

        q = ''.join(indices_char[i] for i in question[0])
        a = ''.join(indices_char[i] for i in answer[0])
        p = ''.join(indices_char[i] for i in prediction[0])

        print('-' * 10)
        print('Q:  ', q)
        print('A:  ', p)
        print('T/F:', end=' ')
        if a == p:
            print('T')
        else:
            print('F')
    print('-' * 10 + '\n')

Train on 18000 samples, validate on 2000 samples
Epoch 1/20
18000/18000 [==============================] - 9s 526us/step - loss: 2.0351 - acc: 0.2910 - val_loss: 1.8399 - val_acc: 0.3491
Epoch 2/20
18000/18000 [==============================] - 9s 519us/step - loss: 1.7966 - acc: 0.3526 - val_loss: 1.7778 - val_acc: 0.3485
Epoch 3/20
18000/18000 [==============================] - 10s 535us/step - loss: 1.7718 - acc: 0.3525 - val_loss: 1.7612 - val_acc: 0.3547
Epoch 4/20
18000/18000 [==============================] - 10s 543us/step - loss: 1.7508 - acc: 0.3583 - val_loss: 1.7494 - val_acc: 0.3627
Epoch 5/20
18000/18000 [==============================] - 10s 553us/step - loss: 1.7243 - acc: 0.3623 - val_loss: 1.7143 - val_acc: 0.3736
Epoch 6/20
18000/18000 [==============================] - 9s 516us/step - loss: 1.6970 - acc: 0.3724 - val_loss: 1.6995 - val_acc: 0.3860
Epoch 7/20
18000/18000 [==============================] - 9s 525us/step - loss: 1.6697 - acc: 0.3829 - val_loss: 1.6528 

Epoch 13/20
18000/18000 [==============================] - 9s 514us/step - loss: 0.2104 - acc: 0.9451 - val_loss: 0.2998 - val_acc: 0.8963
Epoch 14/20
18000/18000 [==============================] - 9s 516us/step - loss: 0.2003 - acc: 0.9486 - val_loss: 0.2860 - val_acc: 0.9045
Epoch 15/20
18000/18000 [==============================] - 9s 506us/step - loss: 0.1931 - acc: 0.9503 - val_loss: 0.2702 - val_acc: 0.9096
Epoch 16/20
18000/18000 [==============================] - 9s 507us/step - loss: 0.1773 - acc: 0.9571 - val_loss: 0.2625 - val_acc: 0.9149
Epoch 17/20
18000/18000 [==============================] - 10s 528us/step - loss: 0.1696 - acc: 0.9604 - val_loss: 0.2554 - val_acc: 0.9135
Epoch 18/20
18000/18000 [==============================] - 9s 518us/step - loss: 0.1589 - acc: 0.9633 - val_loss: 0.2558 - val_acc: 0.9131
Epoch 19/20
18000/18000 [==============================] - 9s 508us/step - loss: 0.1610 - acc: 0.9614 - val_loss: 0.3195 - val_acc: 0.8849
Epoch 20/20
18000/18000 [=

18000/18000 [==============================] - 9s 514us/step - loss: 0.0037 - acc: 1.0000 - val_loss: 0.1522 - val_acc: 0.9551
Epoch 14/20
18000/18000 [==============================] - 10s 539us/step - loss: 0.0035 - acc: 1.0000 - val_loss: 0.1524 - val_acc: 0.9556
Epoch 15/20
18000/18000 [==============================] - 9s 509us/step - loss: 0.0034 - acc: 1.0000 - val_loss: 0.1535 - val_acc: 0.9562
Epoch 16/20
18000/18000 [==============================] - 10s 546us/step - loss: 0.0033 - acc: 1.0000 - val_loss: 0.1529 - val_acc: 0.9562
Epoch 17/20
18000/18000 [==============================] - 9s 528us/step - loss: 0.0032 - acc: 1.0000 - val_loss: 0.1568 - val_acc: 0.9558
Epoch 18/20
18000/18000 [==============================] - 9s 500us/step - loss: 0.0031 - acc: 1.0000 - val_loss: 0.1545 - val_acc: 0.9559
Epoch 19/20
18000/18000 [==============================] - 10s 564us/step - loss: 0.0030 - acc: 1.0000 - val_loss: 0.1543 - val_acc: 0.9559
Epoch 20/20
18000/18000 [===========

In [7]:
model.save("addproblem.h5")

In [8]:
ls

Overview.ipynb  README.md  Sample1.ipynb  Sample3.ipynb  addproblem.h5
